🌡️ Temperature Classification using DHT22
Classes: Cold – Warm – Hot
1️⃣ Problem Definition (Very Important)
Sensor

DHT22 temperature sensor

Output: Temperature in °C (continuous value)

Goal

Convert a continuous temperature into discrete classes:Cold

Warm

Hot

👉 This is a CLASSIFICATION problem, not regression.

2️⃣ Define the Classes (Domain Knowledge)

You must define ranges first (this is important for teaching).

Example class definition
Class	Temperature Range (°C)
Cold	< 20
Warm	20 – 30
Hot	> 30

📌 You can adjust these ranges based on location.

3️⃣ Dataset Creation (Simple & Realistic)
Example dataset format
Temperature (°C)	Label
16.5	Cold
22.0	Warm
28.5	Warm
32.0	Hot
35.5	Hot

Python label encoding

In [3]:
def label_temperature_norm(temp_norm):
    temp = temp_norm * 50.0  # convert back to °C
    if temp < 20:
        return 0   # Cold
    elif temp <= 30:
        return 1   # Warm
    else:
        return 2   # Hot


Class mapping:

In [4]:
class_names = ["Cold", "Warm", "Hot"]

Create Training Data (Synthetic or Real)

In [12]:
import numpy as np
import sklearn

# Generate synthetic temperature data
temps = np.random.uniform(10, 40, 600)

# Normalize (DHT22 max range ≈ 50°C)
temps_norm = temps / 50.0

labels = np.array([label_temperature_norm(t) for t in temps_norm])

X = temps_norm.reshape(-1, 1)
y = labels


⚠️ Golden Rule:

Same normalization must be applied on STM32 / ESP32.

Check class balance

In [13]:
unique, counts = np.unique(y, return_counts=True)
class_distribution = dict(zip(unique, counts))
print(class_distribution)

{np.int64(0): np.int64(207), np.int64(1): np.int64(193), np.int64(2): np.int64(200)}



5️⃣ Split Data (Train / Test)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

Check class balance twice (industry practice)

1️⃣ After label creation (full dataset)
2️⃣ After train-test split

In [15]:
# Training set balance
np.unique(y_train, return_counts=True)

# Test set balance
np.unique(y_test, return_counts=True)

(array([0, 1, 2]), array([33, 48, 39]))

6️⃣ Build Embedded-Friendly Model

Minimal neural network (perfect for MCUs):

In [16]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(1,)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])


model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 16)             │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 195 (780.00 B)

 Trainable params: 195 (780.00 B)

 Non-trainable params: 0 (0.00 B)

7️⃣ Train the Model

In [17]:
history = model.fit(
    X_train, y_train,
    epochs=150,
    batch_size=16,
    validation_split=0.1,
    verbose=1
)

Epoch 1/150
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3449 - loss: 1.0866 - val_accuracy: 0.2500 - val_loss: 1.0907
Epoch 2/150
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3449 - loss: 1.0724 - val_accuracy: 0.2500 - val_loss: 1.0842
Epoch 3/150
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3449 - loss: 1.0575 - val_accuracy: 0.2500 - val_loss: 1.0730
Epoch 4/150
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3519 - loss: 1.0415 - val_accuracy: 0.3958 - val_loss: 1.0543
Epoch 5/150
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4583 - loss: 1.0215 - val_accuracy: 0.5000 - val_loss: 1.0337
Epoch 6/150
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5417 - loss: 0.9984 - val_accuracy: 0.5000 - val_loss: 1.0110
Epoch 7/150
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5556 - loss: 0.9722 - val_accuracy: 0.5000 - val_loss: 0.9860
Epoch 8/150
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5903 - loss: 0.9433 - val_accuracy: 0.6250 - v

8️⃣ Evaluate the Model

In [18]:
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9917 - loss: 0.1105  
Test Accuracy: 0.9916666746139526


9️⃣ Test with Known Temperatures (Demo Cell)

In [23]:
test_temps = np.array([21, 42, 35]).reshape(-1, 1)
test_temps_norm = test_temps / 50.0

pred = model.predict(test_temps_norm)
pred_classes = np.argmax(pred, axis=1)

for t, c in zip(test_temps.flatten(), pred_classes):
    print(f"{t} °C → {class_names[c]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
21 °C → Warm
42 °C → Hot
35 °C → Hot


🔟 Confidence Interpretation (Very Important)

In [21]:
confidence = np.max(pred, axis=1)

for t, conf in zip(test_temps.flatten(), confidence):
    print(f"Confidence: {conf:.2f}")


Confidence: 1.00
Confidence: 1.00
Confidence: 1.00


1️⃣1️⃣ Convert to TensorFlow Lite (For Embedded)

In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

with open("dht22_temp_classifier.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model size:", len(tflite_model), "bytes")


INFO:tensorflow:Assets written to: C:\Users\z665059\AppData\Local\Temp\tmptg6sbzax\assets


INFO:tensorflow:Assets written to: C:\Users\z665059\AppData\Local\Temp\tmptg6sbzax\assets


Saved artifact at 'C:\Users\z665059\AppData\Local\Temp\tmptg6sbzax'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2577736925456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2577736268688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2577736926224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2577736925840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2577736926608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2577736927952: TensorSpec(shape=(), dtype=tf.resource, name=None)
TFLite model size: 2880 bytes
